In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option('display.max_columns', 999)

In [ ]:
df = pd.read_csv("/home/aris/Coding/contiki-experiments/Untitled 1.csv")
df[14:17]

In [ ]:
df.shape

In [ ]:
list(df.columns)

In [ ]:
input_columns = ['SimDir',
 'DataSendStartTimeInS',
 'DataSendIntervalTimeInS',
 'TschSchedulePrintIntervalInS',
 'WithThunderAdaptiveEbSlot',
 'SimulationEndTimeInS',
 'PowertrackerIntervalTimeInS',
 'PowertrackerInitialDelayTimeInS',
 'RadioQualitySetTimeInS',
 'GrandparentKillTimeInS',
 'GrandchildOfGrandparentKillMoteId',
 'RandomSeedNumber',
 'TschConfMacMaxFrameRetries',
 'LeapfrogRetries',
 'RadioQualitySetQualityInPercent',
 'RadioQualitySetRootQualityInPercent',
 'MakeWithThunder',
 'MakeWithLinkpeek',
 'LinkpeekDefaultMaxParents',
 'MakeWithLeapfrog',
 'WithOverhearing',
 'RadioQualityRandomSetMinQualityInPercent',
 'RadioQualityRandomSetMaxQualityInPercent',
 'RadioQualityRandomStartSetTimeInS',
 'RadioQualityRandomSetIntervalTimeInS']

In [ ]:
output_columns = list(set(df.columns) - set(input_columns))
output_columns

In [ ]:
group_by_columns = list(set(input_columns) -(set(input_columns) - set(df.columns)) -set(['SimDir', 'RandomSeedNumber']))
group_by_columns

In [ ]:
dfs = df.copy()
for column_name in group_by_columns:
    dfs[column_name] = dfs[column_name].apply(str)
dfs.info()

In [ ]:
dfg_grouping = dfs.groupby(group_by_columns)
print(len(dfg_grouping.groups))
aggregation_dict = {
'PostKillSent':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PreKillReceived':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PreKillSent':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PreKillNotReceived':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PreKillPDR':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PostKillReceived':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PreKillReceivedUnique':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PostKillPDR':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PostKillReceivedUnique':['sum', 'mean', 'count', 'std', 'min', 'max'],
 'PostKillNotReceived':['sum', 'mean', 'count', 'std', 'min', 'max']
}

dfg = dfg_grouping.aggregate(aggregation_dict)
#dfg.columns = ['_'.join(tuple(map(str, t))).rstrip('_') for t in dfg.columns.values]
#dfg.columns = dfg.columns.tolist()
dfg.reset_index(inplace=True)
dfg


In [ ]:
dfg.info()


In [ ]:
def makeScenario(row):
    if row['MakeWithLeapfrog'][0] == '1.0':
        if row['TschConfMacMaxFrameRetries'][0] == '1' and row['WithOverhearing'][0] == '1.0':
            return "LFC"
        elif row['TschConfMacMaxFrameRetries'][0] == '1' and row['WithOverhearing'][0]=='nan':
            return "LFC -OH"
        elif row['TschConfMacMaxFrameRetries'][0] == '0' and row['WithOverhearing'][0] == '1.0':
            return "LFC 0RT"
        else:
            return str((row['TschConfMacMaxFrameRetries'][0], row['WithOverhearing'][0]))
    else:
        return "TSCH "+str(row['TschConfMacMaxFrameRetries'][0])+"RT"

In [ ]:
dfg['Scenario'] = dfg.apply(makeScenario, axis=1)
dfg['Scenario']

In [ ]:
lfc_scenarios = ['LFC', 'LFC 0RT','LFC -OH']
tsch_scenarios = ['TSCH 8RT', 'TSCH 6RT', 'TSCH 4RT', 'TSCH 2RT', 'TSCH 0RT']
scenarios = lfc_scenarios + tsch_scenarios
assert(set(scenarios) == set(np.unique(dfg['Scenario'])))
scenarios

In [ ]:
groups = list(reversed(sorted(np.unique(dfg['RadioQualitySetQualityInPercent']))))
groups

In [ ]:
# Setting the positions and width for the bars
single_bar_width_used = 0.20
single_bar_width = single_bar_width_used*1.20 #20% space
group_width_used = len(scenarios)*single_bar_width
group_width = group_width_used*1.20 #20% space
group_width_unused = group_width - group_width_used

# Plotting the bars
fig, ax = plt.subplots(figsize=(10,5))

kept_labels_plots = []
def make_scenario_subgroup(x_offset, group_name, scenario_names, colormap):
    for scenario_index, scenario_name in enumerate(scenario_names):
        x = x_offset + scenario_index*single_bar_width
        y = dfg[(dfg['Scenario'] == scenario_name) &
                (dfg['RadioQualitySetQualityInPercent'] == group_name)
               ][('PreKillPDR', 'mean')]*100
        error = dfg[(dfg['Scenario'] == scenario_name) &
                    (dfg['RadioQualitySetQualityInPercent'] == group_name)
                   ][('PreKillPDR', 'std')]*100
        label = str(scenario_name)
        keptplot = ax.bar(x, 
                y, 
                # of width
                single_bar_width_used, 
                # with alpha 0.5
#                 alpha=0.5, 
                # with color
                color=colormap(float(scenario_index)/len(scenario_names)),
                label=label) 
        ax.errorbar(x, y, yerr=error, ecolor="#0000FF", fmt='-o')
        kept_labels_plots.append((label, keptplot))
    
for group_index, group_name in enumerate(groups):
    group_offset = group_width*group_index
    make_scenario_subgroup(group_offset, group_name, lfc_scenarios, plt.cm.Greens_r)
    make_scenario_subgroup(group_offset+(single_bar_width*len(lfc_scenarios)), group_name, tsch_scenarios, plt.cm.Reds_r)


# Set the y axis label
ax.set_xlabel('Link quality in %')
ax.set_ylabel('Packet Delivery Rate in %')

# Set the chart's title
ax.set_title('Packet Delivery Rate')

# Set the position of the x ticks
ax.set_xticks([group_index*group_width + 0.5 * group_width_used for group_index, group_name in enumerate(groups)])

# Set the labels for the x ticks
ax.set_xticklabels([str(group_name)+"%" for group_name in groups])

# Setting the x-axis and y-axis limits
plt.xlim(-0.5*single_bar_width, group_width*len(groups) - group_width_unused)
plt.ylim([0, max(dfg['PreKillPDR', 'mean']*100+dfg['PreKillPDR', 'std']*100+10)] )

# get handles

kept_labels_plots_dict = dict(kept_labels_plots)

# remove the errorbars
#handles = [h[0] for h in handnano les]
# use them in the legend
# ax1.legend(handles, labels, loc='upper left',numpoints=1)

# Adding the legend and showing the plot
ax.legend([kept_labels_plots_dict[scenario] for scenario in scenarios], scenarios, bbox_to_anchor=(1, 1), numpoints=1)
ax.grid(axis='y')
plt.show()
# plt.savefig('PDR-1000pkts x5.png', dpi=200, bbox_inches='tight')